<a href="https://colab.research.google.com/github/PaulloSergio/Otimizacao/blob/main/Optimization_Bootcamp_Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio 

Considerando um capital para investimento de R\$1.000.000 e as seguintes opções de investimento:

| Opção  | Descrição                                         | Custo do investimento (R\$) | Retorno esperado (R\$) |
|:-------:|---------------------------------------------------|:---------------------------:|:----------------------:|
| 1     | Ampliação da capacidade do armazém ZDP em 5%      |           470.000           |         410.000        |
| 2     | Ampliação da capacidade do armazém MGL em 7%      |           400.000           |         330.000        |
| 3     | Compra de empilhadeira                            |           170.000           |         140.000        |
| 4     | Projeto de P&D I                                  |           270.000           |         250.000        |
| 5     | Projeto de P&D II                                 |           340.000           |         320.000        |
| 6     | Aquisição de novos equipamentos                   |           230.000           |         320.000        |
| 7     | Capacitação de funcionários  |            50.000           |         90.000         |
| 8     | Ampliação da estrutura de carga rodoviária        |           440.000           |         190.000        |

Desenvolva um algoritmo de otimização para selecionar os projetos que maximizam o retorno total esperado, considerando que:

* Se o investimento 1 for selecionado, então o investimento 5 não deve ser;
* Se o investimento 2 for selecionado, então o investimento 4 também deve ser.


Prepare uma apresentação demonstrando os resultados da atividade. Sua apresentação deve ser no formato de **pitch**, deve durar **até 15 minutos**, e deve conter as seguintes seções:
  * "Quem sou eu?": uma breve apresentação sobre você e sua formação;
  * "Qual o problema estou resolvendo?";
  * "Como eu resolvi o problema?";
  * Resultados e Análises.

## Classe de dados

In [76]:
from dataclasses import dataclass

@dataclass
class OpcaoInvestimento:
    descricao: str
    custo: float
    retorno: float

## Transformando os dados em uma tabela

In [77]:
import pandas as pd
from typing import List

def items_to_table(items: List[OpcaoInvestimento]) -> pd.DataFrame:
    records = [{
            'Descrição': i.descricao,
            'Custo ($)': i.custo,
            'Retorno ($)': i.retorno } for i in items]
            
    records.append({
        'Descrição': 'Total',
        'Custo ($)': sum(i.custo for i in items),
        'Retorno ($)': sum(i.retorno for i in items)
    })
    return pd.DataFrame.from_records(records)

In [78]:
# Variávies do problema 

custo_maximo = 1000000

custo = [470000, 400000, 170000, 270000, 340000, 230000, 50000, 440000]

retorno = [410000, 330000, 140000, 250000, 320000, 320000, 90000, 190000]


descricao = ['Ampliação da capacidade do armazém ZDP em 5%',
             'Ampliação da capacidade do armazém MGL em 7%',
             'Compra de empilhadeira', 'Projeto de P&D I',
             'Projeto de P&D II', 'Aquisição de novos equipamentos',
             'Capacitação de funcionários',
             'Ampliação da estrutura de carga rodoviária']

available_items = [OpcaoInvestimento(f'{descricao[i]}', c, r) for i, (c, r) in enumerate(zip(custo, retorno))]

items_to_table(available_items)

,Descrição,Custo ($),Retorno ($)
0,Ampliação da capacidade do armazém ZDP em 5%,470000,410000
1,Ampliação da capacidade do armazém MGL em 7%,400000,330000
2,Compra de empilhadeira,170000,140000
3,Projeto de P&D I,270000,250000
4,Projeto de P&D II,340000,320000
5,Aquisição de novos equipamentos,230000,320000
6,Capacitação de funcionários,50000,90000
7,Ampliação da estrutura de carga rodoviária,440000,190000
8,Total,2370000,2050000


## Modelagem

## Parâmetros 
c = custo 

r = retorno

d = descrição

Se o investimento 1 for selecionado, então o investimento 5 não deve ser;

Se o investimento 2 for selecionado, então o investimento 4 também deve ser.


**Formulação do problema**:

$$ \max   \; retorno $$
$$ \text{subject to: } custototal < 1000000 $$

## Construtor do problema


In [79]:
!pip install science-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [80]:
from science_optimization.builder import (
    BuilderOptimizationProblem,
    Variable,
    Constraint,
    Objective,
    OptimizationProblem
)
from science_optimization.function import (
    FunctionsComposite, 
    LinearFunction,
)
from science_optimization.solvers import Optimizer
from science_optimization.algorithms.linear_programming import Glop
import numpy as np

class Investiment(BuilderOptimizationProblem):
  def __init__( # Metódo construtor
      self,
      custo_maximo: float,
      available_items: List[OpcaoInvestimento]):
    
    self.__custo_maximo = custo_maximo
    self.__items = available_items

  
  @property
  def __num_vars(self) -> int:
    return len(self.__items)

  @property
  def __custo(self) -> np.array:
    return np.array([item.custo for item in self.__items]
                    ).reshape(-1, 1)
  
  @property
  def __retorno(self) -> np.array:
    return np.array([item.retorno for item in self.__items]
                    ).reshape(-1, 1)

  def build_variables(self):
    x_min = np.zeros((self.__num_vars, 1))
    x_max = np.ones((self.__num_vars, 1))
    x_type=['d']*self.__num_vars # Discrete variable
    variables = Variable(x_min, x_max, x_type)
    return variables
  
  def build_constraints(self) -> Constraint:
        """Custos cannot exceed custo_maximo"""
        # c - c_m <= 0
        constraint = LinearFunction(c=self.__custo, d=-self.__custo_maximo)
        ineq_cons = FunctionsComposite()
        ineq_cons.add(constraint)
        constraints =Constraint(ineq_cons=ineq_cons)
       

        return constraints

  def build_objectives(self) -> Objective:
      # minimize -v*x
      obj_fun = LinearFunction(c=-self.__retorno)

      obj_funs = FunctionsComposite()
      obj_funs.add(obj_fun)
      objective = Objective(objective=obj_funs)

      return objective

In [81]:
def optimization_problem(
    custo_maximo: float,
    available_items: List[OpcaoInvestimento],
    verbose: bool = False
) -> OptimizationProblem:
    investiment = Investiment(custo_maximo, available_items)
    problem = OptimizationProblem(builder=investiment)
    if verbose:
        print(problem.info())
    return problem

## Otimização

In [82]:
def run_optimization(
    problem: OptimizationProblem,
    verbose: bool = False,
) -> np.array:
    optimizer = Optimizer(
        opt_problem=problem,
        algorithm=Glop()
    )
    results = optimizer.optimize()
    decision_variables = results.x.ravel()

    def rest(a, b):
      if decision_variables[1] == True and decision_variables[b] == False :
        ret = 2
        j = 0
        retornox = 2000000
        for item, item_was in zip(available_items, decision_variables):
          if item_was:
            x = (item.custo / item.retorno)
            if x <= ret and item.custo >= a  and j!=b and item.retorno < retornox:
              retornox = item.retorno
              z = j
              j += 1
            else:
              j += 1
          else:
            j += 1    
        decision_variables[z] = 0
        decision_variables[3] = 1
    rest(270000, 3)
    if verbose:
      print(f"Decision variable:{decision_variables}")
    return decision_variables

In [83]:
def investiment_milp(
    custo_maximo: float, 
    items: List[OpcaoInvestimento],
    verbose:bool = False) -> List[OpcaoInvestimento]:
    
    problem = optimization_problem(custo_maximo, 
                                   available_items, 
                                   verbose)
    decision_variables = run_optimization(problem, verbose,)
    
    if decision_variables[0] == True and decision_variables[4] == True:
      for item, item_was in zip(available_items, decision_variables):
        if available_items[0]:
          x = (item.custo / item.retorno)
        if available_items[4]:
          y = (item.custo / item.retorno)
      if x >= y:
        available_items.pop(4)
        problem = optimization_problem(custo_maximo, 
                                   available_items, 
                                   verbose)
        decision_variables = run_optimization(problem, verbose)
      else:
        available_items.pop(0)
        problem = optimization_problem(custo_maximo, 
                                   available_items, 
                                   verbose)
        decision_variables = run_optimization(problem, verbose)
    
    
    # Build list of chosen items
    chosen_items = list()
    for item, item_was_chosen in zip(available_items, decision_variables):
        if item_was_chosen:
          chosen_items.append(item)
    return chosen_items

In [84]:
chosen_items = investiment_milp(custo_maximo, available_items, verbose=True)
items_to_table(chosen_items)



Numbers of objectives: 1
Linear objective coefficients (c'x):
c =
[[-410000 -330000 -140000 -250000 -320000 -320000  -90000 -190000]]
Numbers of variables: 8
	 [0.] <= x_0 <= [1.]
	 [0.] <= x_1 <= [1.]
	 [0.] <= x_2 <= [1.]
	 [0.] <= x_3 <= [1.]
	 [0.] <= x_4 <= [1.]
	 [0.] <= x_5 <= [1.]
	 [0.] <= x_6 <= [1.]
	 [0.] <= x_7 <= [1.]
Inequality Linear Constraints (A*x <= b):
A =
[[470000 400000 170000 270000 340000 230000  50000 440000]]
b =
[[1000000]]
None
Decision variable:[0. 1. 0. 1. 0. 1. 1. 0.]


,Descrição,Custo ($),Retorno ($)
0,Ampliação da capacidade do armazém MGL em 7%,400000,330000
1,Projeto de P&D I,270000,250000
2,Aquisição de novos equipamentos,230000,320000
3,Capacitação de funcionários,50000,90000
4,Total,950000,990000
